<div class="alert alert-block alert-success">
    
# FIT5196 Task 1 in Assessment 1
#### Student Name: Robiatul Adawiyah Al-Qosh
#### Student ID: 34269193

Date: 14/10/2024


Environment: Python 3.10

Libraries used:
* re (for regular expression, installed and imported)
* pandas (for data manipulation)
* os (for acces files in folder)
* datetime (for datetime converting)
* json (for creating JSON file)


    
</div>

<div class="alert alert-block alert-danger">
    
## Table of Contents

</div>    

[1. Introduction](#Intro) <br>
[2. Importing Libraries](#libs) <br>
[3. Examining Patent Files](#examine) <br>
[4. Loading and Parsing Files](#load) <br>
$\;\;\;\;$[4.1. Defining Regular Expressions](#Reg_Exp) <br>
$\;\;\;\;$[4.2. Reading Files](#Read) <br>
$\;\;\;\;$[4.3. Whatever else](#latin) <br>
[5. Writing to CSV/JSON File](#write) <br>
$\;\;\;\;$[5.1. Verification - using the sample files](#test_xml) <br>
[6. Summary](#summary) <br>
[7. References](#Ref) <br>

-------------------------------------

<div class="alert alert-block alert-warning">

## 1.  Introduction  <a class="anchor" name="Intro"></a>
    
</div>

This assessment regards extracting data from semi-sctuctured text files. The dataset contained 15 txt files and 1 Excel file (contain 16 sheets) which included various information about user reviews. In particular, the txt files store attributes like user_id, gmap_id, date, rating, review_text, etc., while the Excel sheets provide metadata. The goal is to parse, clean, and merge the data by normalizing timestamps and ensuring consistent tag structures.

-------------------------------------

<div class="alert alert-block alert-warning">
    
## 2.  Importing Libraries  <a class="anchor" name="libs"></a>
 </div>

The packages to be used in this assessment are imported in the following. They are used to fulfill the following tasks:

* **re:** to define and use regular expressions
* **pandas:** to do data manipulation
* **os:** to access all the necessary file at once
* **datetime:** to convert timestamp in time tag
* **json:** to create JSON file

In [367]:
import re
import pandas as pd
import os
from datetime import datetime
import json

In [368]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


-------------------------------------

<div class="alert alert-block alert-warning">

## 3.  Examining Raw Data <a class="anchor" name="examine"></a>

 </div>

First of all, set the working directory that contains all the source files.

In [369]:
student_008 = "/content/drive/MyDrive/S2 Monash University/MUI Y2 T1/ITI5196 Data Wrangling/Assignment 1/student_008"
# Plese adjust the path

Look all the tags in the txt files.



In [370]:
# Using regex to see all the tags
tag_pattern = re.compile(r"<(.*?)>") # Tags are inside the "<>"

# Create a function to extract the tags from the file
def extract_tags_from_file(filepath):
    with open(filepath, "r", encoding="utf-8") as file:
        content = file.read()
    tags = tag_pattern.findall(content)
    return tags

# Loop for all txt file inside the working directory
all_tags = set()  # Using set to avoid duplicate
for filename in os.listdir(student_008):
    if filename.endswith(".txt"):
        file_tags = extract_tags_from_file(os.path.join(student_008, filename))
        all_tags.update(file_tags)

# Show all tags
print("All Tags:")
print(sorted(all_tags))

All Tags:
[' /Date', ' /GmapID', ' /Gmap_id', ' /Name', ' /Pics', ' /Pictures', ' /Rate', ' /Rating', ' /Resp', ' /Response', ' /Review', ' /Text', ' /Time', ' /UserId.', ' /date', ' /gmapID', ' /gmap_id', ' /name', ' /pics', ' /pictures', ' /rate', ' /rating', ' /resp', ' /response', ' /review', ' /text', ' /time', ' /user', ' /user_id', ' /user_name', ' /userid', ' /username', ' Date', ' GmapID', ' Gmap_id', ' Name', ' Pics', ' Pictures', ' Rate', ' Rating', ' Resp', ' Response', ' Review', ' Text', ' Time', ' UserId.', ' date', ' gmapID', ' gmap_id', ' name', ' pics', ' pictures', ' rate', ' rating', ' resp', ' response', ' review', ' text', ' time', ' user', ' user_id', ' user_name', ' userid', ' username', '$20).  Good service with charming location, especially on the patio.< /Text', '//Date', '//GmapID', '//Gmap_id', '//Name', '//Pics', '//Pictures', '//Rate', '//Rating', '//Resp', '//Response', '//Review', '//Text', '//Time', '//UserId.', '//date', '//gmapID', '//gmap_id', '//na

Count the number of data that is recorded inside the txt files.

In [371]:
# Create a function to count tags in a file
def count_tags_in_file(filepath, tags):
    counts = {tag: 0 for tag in tags}
    with open(filepath, "r", encoding="utf-8") as file:
        content = file.read()
    for tag in tags:
        pattern = re.compile(fr"<({tag})>")
        counts[tag] += len(pattern.findall(content))
    return counts

# Create a function that count the total tags from all tha file
def count_tags_in_all_files(tags, folder):
    total_counts = {tag: 0 for tag in tags}
    for filename in os.listdir(folder):
        if filename.endswith(".txt"):
            filepath = os.path.join(folder, filename)
            file_counts = count_tags_in_file(filepath, tags)
            for tag, count in file_counts.items():
                total_counts[tag] += count
    return total_counts

# Fill the tags with record that needed (the tags can be added with tag needed)
tags = ["record"]
total_counts = count_tags_in_all_files(tags, student_008)

# Show the total tags
print("\nTotal data recorded in txt files:")
for tag, total in total_counts.items():
    print(f"  <{tag}>: {total}")


Total data recorded in txt files:
  <record>: 33782


Having examined the file content, the following observations were made:

*   There are inconsistencies in tags that used in txt files. So, the appropriate regex must be used due to include the intended tags.
*   The number of data that is recorded in the txt files is 33,782 based on the total of record tag.





-------------------------------------

<div class="alert alert-block alert-warning">

## 4.  Loading and Parsing Files <a class="anchor" name="load"></a>

</div>

In this section, the files are parsed and processed. First of all, appropriate regular expressions are defined to extract desired information when reading the files. The focus is on handling variations in tag structures and inconsistencies, such as differences in capitalization or spacing. Each file is read sequentially, and relevant attributes like gmap_id, user_id, time, rating, text, pics, and resp are extracted using the defined regex patterns. The extracted data is then stored in a structured list for further processing.

-------------------------------------

<div class="alert alert-block alert-info">
    
### 4.1. Defining Regular Expressions <a class="anchor" name="Reg_Exp"></a>

Defining correct regular expressions is crucial in extracting desired information from the text efficiently. The regular expressions are designed to handle minor inconsistencies across the files, such as optional spaces, mixed-case tags, or multiple closing slashes. For example, patterns for \<user_id> and \<UserId> are unified using flexible regex patterns to ensure both formats are captured. Additionally, backreferences are used to ensure matching pairs of opening and closing tags, preventing extraction errors. The patterns are applied with case-insensitive matching to ensure robustness and accuracy in parsing.

In [372]:
pattern_id = re.compile(  #reg ex pattern or id
    r"<\s*[Gg]map_?[Ii][Dd]\s*>(.*?)<.*\/*[Gg]map_?[Ii][Dd]\s*>|"           # gmap_id
    r"<\s*[Uu]ser_?I?i?d?\.?\s*>(.*?)<.*\/*[Uu]ser_?I?i?d?\.?\s*/*>|"       # user_id
    r"<\s*[DdTt][ai][tm]e\s*>\s*(\d+)?\s*<.*\/*[DdTt][ai][tm]e\s*>|"        # time
    r"<\s*[Rr]ate?[ing]*\s*>(.*?)<.*\/*[Rr]ate?[ing]*\s*>|"                 # rating
    r"<\s*[RrTt]e[vx][it][iew]*\s*>(.*?)<.*\/*[RrTt]e[vx][it][iew]*\s*>|"   # text (review)
    r"<\s*[Pp]ics?[tures]*\s*>(.*?)<.*\/*[Pp]ics?[tures]*\s*>|"             # pics
    r"<\s*[Rr]esp[onse]*\s*>(.*?)<.*\/*[Rr]esp[onse]*\s*\/?>",              # resp
    re.DOTALL)

These patterns are used in the next step when reading the files.

-------------------------------------

<div class="alert alert-block alert-info">
    
### 4.2. Reading Files <a class="anchor" name="Read"></a>

In this step, all files are read and parsed. The first files to be handled are the txt files.

In [373]:
# Create the blank list for the data form the txt files
txt_data = []

# Create a function to convert the timestamp in time data
def convert_timestamp(timestamp):
    if timestamp:  # Pastikan timestamp tidak kosong
        return datetime.utcfromtimestamp(int(timestamp) / 1000).strftime('%Y-%m-%d %H:%M:%S')
    return None

# Loop to process all the txt files in the working directory
for filename in os.listdir(student_008):
    if filename.endswith(".txt"):
        with open(os.path.join(student_008, filename), "r", encoding="utf-8") as file:
            content = file.read()

        # Split the data based on <record>...</record>
        records = re.split(r"<\s*record[^>]*>.*?", content, flags=re.DOTALL)

        # Extrac data in every record with previous regex pattern
        for record in records:
            matches = pattern_id.findall(record)

            if matches:
                # Create the blank dictionary for every necessary tags
                record_data = {
                    "gmap_id": None,
                    "user_id": None,
                    "time": None,
                    "rating": None,
                    "text": None,
                    "pics": None,
                    "resp": None
                }

                # Fill the data from files
                for match in matches:
                    gmap_id, user_id, time, rating, text, pics, resp = match
                    if gmap_id:
                        record_data["gmap_id"] = gmap_id.strip()
                    if user_id:
                        record_data["user_id"] = user_id.strip()
                    if time:
                        record_data["time"] = convert_timestamp(time)
                    if rating:
                        record_data["rating"] = int(rating)
                    if text:
                        record_data["text"] = text.strip()
                    if pics:
                        record_data["pics"] = pics.strip()
                    if resp:
                        record_data["resp"] = resp.strip()

                txt_data.append(record_data)

Let's take a look at the first ten elements of the lists generated. We can see that ids, reviews,etc. are parsed and stored correctly.

In [374]:
# See the first 10 data from the txt files
txt_data[:10]

[{'gmap_id': '0x808e32803ca5056d:0x1748a17978e712a3',
  'user_id': '116363567657011126649',
  'time': '2021-08-21 20:01:03',
  'rating': 5,
  'text': "I've bought paints here and there coming back every other day recently and I've found that Liz, Hector, Alex and Ramon (Apologies if I misspelled the name) are very helpful and provide great customer service. The idea is they're all trying to do best on their job and some customers are entitled. I, as customer, treat these folks with respect, hence, I've gotten it back. Customer service at this store has changed for the better, thanks to everyone's effort at this Kelly Moore store, since the last couple of years, and like to come back whenever I need paint as opposed to before that I'd  go to this store only and only if I have to.",
  'pics': 'None',
  'resp': "{'time': 1629747779487, 'text': 'Dear Frederick Nguyen, Thank you for your wonderful review on our Kelly-Moore Paint store!  We are very happy we could help.'}"},
 {'gmap_id': '0x

-------------------------------------

<div class="alert alert-block alert-info">
    
### 4.3. Whatever else <a class="anchor" name="latin"></a>

the rest of your methodology

Create a new data frame contians the data from txt files.

In [375]:
# Chenge list to data frame
df_txt = pd.DataFrame(txt_data)

# Show the data
df_txt

,gmap_id,user_id,time,rating,text,pics,resp
0,0x808e32803ca5056d:0x1748a17978e712a3,116363567657011126649,2021-08-21 20:01:03,5,I've bought paints here and there coming back ...,None,"{'time': 1629747779487, 'text': 'Dear Frederic..."
1,0x808e32803ca5056d:0x1748a17978e712a3,109312257578567385533,2021-07-21 06:53:03,5,Liz was very good salesperson...patient and he...,None,"{'time': 1626990530004, 'text': 'Dear Frank C ..."
2,0x808e32803ca5056d:0x1748a17978e712a3,118432808700092375314,2021-07-01 17:47:57,5,I was able to find the paint chips I wanted.,None,"{'time': 1625697160077, 'text': 'Dear Lisa Han..."
3,0x808e32803ca5056d:0x1748a17978e712a3,118011538546540420445,2020-12-10 01:49:29,5,I go there every day as a contractor my time i...,None,"{'time': 1607731545910, 'text': 'Dear John Can..."
4,0x808e32803ca5056d:0x1748a17978e712a3,103167319985107391775,2021-08-13 23:01:15,5,High quality paints,None,"{'time': 1629222927133, 'text': 'Dear jose san..."
...,...,...,...,...,...,...,...
33777,0x80deaadefc6abef7:0x4a64d785b6c70207,109917992905615904886,2020-10-03 05:28:58,5,None,None,None
33778,0x80deaadefc6abef7:0x4a64d785b6c70207,117595977088883520514,2019-01-27 02:00:55,5,None,None,None
33779,0x80deaadefc6abef7:0x4a64d785b6c70207,109946993500775059273,2019-09-01 20:46:19,5,None,None,None
33780,0x80deaadefc6abef7:0x4a64d785b6c70207,116680252402141570684,2018-07-16 21:43:16,4,None,None,None


After the txt data is done and in a data frame form, the Excel file can be handled.

In [376]:
# Read the excel file from the working directory
excel_data = pd.read_excel(os.path.join(student_008, "group008.xlsx"), sheet_name = None)

# Join all the table in all sheet based on the header
df_excel = pd.concat(excel_data.values(), ignore_index = True)

# Show the data
df_excel

,x2,user_id,name,time,rating,text,pics,resp,gmap_id,x0,x4,x3,x1
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,106209248371445133375,Jimmie Cox,1.620335e+12,5.0,I love this Subway. Everyone here is very frie...,NaN,NaN,0x80c22c5b33373b7b:0x65448f6034d1a2b5,NaN,NaN,NaN,NaN
3,NaN,108644153241460415220,화난고양,1.614581e+12,5.0,Good place for quick meal or take out. Order s...,[{'url': ['https://lh5.googleusercontent.com/p...,NaN,0x80c22c5b33373b7b:0x65448f6034d1a2b5,NaN,NaN,NaN,NaN
4,NaN,114363743874334610326,Layne Buck,1.615220e+12,5.0,Superb vibes and very welcoming service. The m...,NaN,NaN,0x80c22c5b33373b7b:0x65448f6034d1a2b5,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2268,NaN,109708230974290243002,Ana Esparza,1.552602e+12,5.0,NaN,NaN,"{'time': 1552660537773, 'text': ""Ana Thank yo...",0x80c2b7dd2ce8c6cb:0x9d551bcaed68b6c0,NaN,NaN,NaN,NaN
2269,NaN,101706798456447612925,Jose Rodriguez,1.611621e+12,3.0,NaN,NaN,"{'time': 1611623931823, 'text': 'Jose it looks...",0x80c2b7dd2ce8c6cb:0x9d551bcaed68b6c0,NaN,NaN,NaN,NaN
2270,NaN,115802745896517675789,Lalo Elias,1.617484e+12,5.0,NaN,NaN,"{'time': 1617715262862, 'text': ""Lalo Thank yo...",0x80c2b7dd2ce8c6cb:0x9d551bcaed68b6c0,NaN,NaN,NaN,NaN
2271,NaN,101780044382660069981,Barry Bryant,1.573343e+12,4.0,NaN,NaN,"{'time': 1573483390798, 'text': 'Barry So clo...",0x80c2b7dd2ce8c6cb:0x9d551bcaed68b6c0,NaN,NaN,NaN,NaN


Remove the unnecessary columns such as x0, x1, etc. and the name column, since it is not needed and do not match the txt data and assignment required file. Then, remove the extra rows that contains NaN or blank in all the cells.

In [377]:
# Drop all the unnecessary columns and the name column to fit the data frome txt files
df_excel = df_excel.drop(columns = ["x0", "x1", "x2", "x3", "x4", "name"])

# Delete the all NaN rows
df_excel = df_excel.dropna(how = "all")

# Show the data
df_excel

,user_id,time,rating,text,pics,resp,gmap_id
2,106209248371445133375,1.620335e+12,5.0,I love this Subway. Everyone here is very frie...,NaN,NaN,0x80c22c5b33373b7b:0x65448f6034d1a2b5
3,108644153241460415220,1.614581e+12,5.0,Good place for quick meal or take out. Order s...,[{'url': ['https://lh5.googleusercontent.com/p...,NaN,0x80c22c5b33373b7b:0x65448f6034d1a2b5
4,114363743874334610326,1.615220e+12,5.0,Superb vibes and very welcoming service. The m...,NaN,NaN,0x80c22c5b33373b7b:0x65448f6034d1a2b5
5,106461781984060872601,1.621998e+12,2.0,I called to ask if they had subway pizza and t...,NaN,NaN,0x80c22c5b33373b7b:0x65448f6034d1a2b5
6,108723819151915883096,1.617908e+12,1.0,I’ve never had a problem with Subway's custome...,NaN,NaN,0x80c22c5b33373b7b:0x65448f6034d1a2b5
...,...,...,...,...,...,...,...
2268,109708230974290243002,1.552602e+12,5.0,NaN,NaN,"{'time': 1552660537773, 'text': ""Ana Thank yo...",0x80c2b7dd2ce8c6cb:0x9d551bcaed68b6c0
2269,101706798456447612925,1.611621e+12,3.0,NaN,NaN,"{'time': 1611623931823, 'text': 'Jose it looks...",0x80c2b7dd2ce8c6cb:0x9d551bcaed68b6c0
2270,115802745896517675789,1.617484e+12,5.0,NaN,NaN,"{'time': 1617715262862, 'text': ""Lalo Thank yo...",0x80c2b7dd2ce8c6cb:0x9d551bcaed68b6c0
2271,101780044382660069981,1.573343e+12,4.0,NaN,NaN,"{'time': 1573483390798, 'text': 'Barry So clo...",0x80c2b7dd2ce8c6cb:0x9d551bcaed68b6c0


Change the data type in time column into proper date time.

In [378]:
# Convert time column to date time data type
df_excel["time"] = pd.to_datetime(df_excel["time"], unit = "ms")

# Show the data
df_excel

,user_id,time,rating,text,pics,resp,gmap_id
2,106209248371445133375,2021-05-06 21:01:54.658,5.0,I love this Subway. Everyone here is very frie...,NaN,NaN,0x80c22c5b33373b7b:0x65448f6034d1a2b5
3,108644153241460415220,2021-03-01 06:39:21.235,5.0,Good place for quick meal or take out. Order s...,[{'url': ['https://lh5.googleusercontent.com/p...,NaN,0x80c22c5b33373b7b:0x65448f6034d1a2b5
4,114363743874334610326,2021-03-08 16:06:19.576,5.0,Superb vibes and very welcoming service. The m...,NaN,NaN,0x80c22c5b33373b7b:0x65448f6034d1a2b5
5,106461781984060872601,2021-05-26 02:54:26.334,2.0,I called to ask if they had subway pizza and t...,NaN,NaN,0x80c22c5b33373b7b:0x65448f6034d1a2b5
6,108723819151915883096,2021-04-08 18:50:40.468,1.0,I’ve never had a problem with Subway's custome...,NaN,NaN,0x80c22c5b33373b7b:0x65448f6034d1a2b5
...,...,...,...,...,...,...,...
2268,109708230974290243002,2019-03-14 22:27:34.475,5.0,NaN,NaN,"{'time': 1552660537773, 'text': ""Ana Thank yo...",0x80c2b7dd2ce8c6cb:0x9d551bcaed68b6c0
2269,101706798456447612925,2021-01-26 00:37:22.816,3.0,NaN,NaN,"{'time': 1611623931823, 'text': 'Jose it looks...",0x80c2b7dd2ce8c6cb:0x9d551bcaed68b6c0
2270,115802745896517675789,2021-04-03 21:12:11.894,5.0,NaN,NaN,"{'time': 1617715262862, 'text': ""Lalo Thank yo...",0x80c2b7dd2ce8c6cb:0x9d551bcaed68b6c0
2271,101780044382660069981,2019-11-09 23:49:11.264,4.0,NaN,NaN,"{'time': 1573483390798, 'text': 'Barry So clo...",0x80c2b7dd2ce8c6cb:0x9d551bcaed68b6c0


Combine all the data both from txt files and excel file together in one data frame.

In [379]:
# Join the txt data frame and excel data frame
df_all = pd.concat([df_txt, df_excel], ignore_index = True)

# Show the data
df_all

,gmap_id,user_id,time,rating,text,pics,resp
0,0x808e32803ca5056d:0x1748a17978e712a3,116363567657011126649,2021-08-21 20:01:03,5.0,I've bought paints here and there coming back ...,None,"{'time': 1629747779487, 'text': 'Dear Frederic..."
1,0x808e32803ca5056d:0x1748a17978e712a3,109312257578567385533,2021-07-21 06:53:03,5.0,Liz was very good salesperson...patient and he...,None,"{'time': 1626990530004, 'text': 'Dear Frank C ..."
2,0x808e32803ca5056d:0x1748a17978e712a3,118432808700092375314,2021-07-01 17:47:57,5.0,I was able to find the paint chips I wanted.,None,"{'time': 1625697160077, 'text': 'Dear Lisa Han..."
3,0x808e32803ca5056d:0x1748a17978e712a3,118011538546540420445,2020-12-10 01:49:29,5.0,I go there every day as a contractor my time i...,None,"{'time': 1607731545910, 'text': 'Dear John Can..."
4,0x808e32803ca5056d:0x1748a17978e712a3,103167319985107391775,2021-08-13 23:01:15,5.0,High quality paints,None,"{'time': 1629222927133, 'text': 'Dear jose san..."
...,...,...,...,...,...,...,...
36011,0x80c2b7dd2ce8c6cb:0x9d551bcaed68b6c0,109708230974290243002,2019-03-14 22:27:34.475000,5.0,NaN,NaN,"{'time': 1552660537773, 'text': ""Ana Thank yo..."
36012,0x80c2b7dd2ce8c6cb:0x9d551bcaed68b6c0,101706798456447612925,2021-01-26 00:37:22.816000,3.0,NaN,NaN,"{'time': 1611623931823, 'text': 'Jose it looks..."
36013,0x80c2b7dd2ce8c6cb:0x9d551bcaed68b6c0,115802745896517675789,2021-04-03 21:12:11.894000,5.0,NaN,NaN,"{'time': 1617715262862, 'text': ""Lalo Thank yo..."
36014,0x80c2b7dd2ce8c6cb:0x9d551bcaed68b6c0,101780044382660069981,2019-11-09 23:49:11.264000,4.0,NaN,NaN,"{'time': 1573483390798, 'text': 'Barry So clo..."


There are inconsistencies in blank/missing values, so standardization is carried out. The decision to keep missing values reamin blank instead of filling it in was made to maintain the authenticity of the data.

In [380]:
# Change the None values from the txt data into <NA>
df_all = df_all.replace("None", pd.NA)

# Change the NaN values from the excel data into <NA>
df_all = df_all.astype("object").where(pd.notna(df_all), pd.NA)

df_all

,gmap_id,user_id,time,rating,text,pics,resp
0,0x808e32803ca5056d:0x1748a17978e712a3,116363567657011126649,2021-08-21 20:01:03,5.0,I've bought paints here and there coming back ...,<NA>,"{'time': 1629747779487, 'text': 'Dear Frederic..."
1,0x808e32803ca5056d:0x1748a17978e712a3,109312257578567385533,2021-07-21 06:53:03,5.0,Liz was very good salesperson...patient and he...,<NA>,"{'time': 1626990530004, 'text': 'Dear Frank C ..."
2,0x808e32803ca5056d:0x1748a17978e712a3,118432808700092375314,2021-07-01 17:47:57,5.0,I was able to find the paint chips I wanted.,<NA>,"{'time': 1625697160077, 'text': 'Dear Lisa Han..."
3,0x808e32803ca5056d:0x1748a17978e712a3,118011538546540420445,2020-12-10 01:49:29,5.0,I go there every day as a contractor my time i...,<NA>,"{'time': 1607731545910, 'text': 'Dear John Can..."
4,0x808e32803ca5056d:0x1748a17978e712a3,103167319985107391775,2021-08-13 23:01:15,5.0,High quality paints,<NA>,"{'time': 1629222927133, 'text': 'Dear jose san..."
...,...,...,...,...,...,...,...
36011,0x80c2b7dd2ce8c6cb:0x9d551bcaed68b6c0,109708230974290243002,2019-03-14 22:27:34.475000,5.0,<NA>,<NA>,"{'time': 1552660537773, 'text': ""Ana Thank yo..."
36012,0x80c2b7dd2ce8c6cb:0x9d551bcaed68b6c0,101706798456447612925,2021-01-26 00:37:22.816000,3.0,<NA>,<NA>,"{'time': 1611623931823, 'text': 'Jose it looks..."
36013,0x80c2b7dd2ce8c6cb:0x9d551bcaed68b6c0,115802745896517675789,2021-04-03 21:12:11.894000,5.0,<NA>,<NA>,"{'time': 1617715262862, 'text': ""Lalo Thank yo..."
36014,0x80c2b7dd2ce8c6cb:0x9d551bcaed68b6c0,101780044382660069981,2019-11-09 23:49:11.264000,4.0,<NA>,<NA>,"{'time': 1573483390798, 'text': 'Barry So clo..."


Next, the information of the data need to be known to run another process.

In [381]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36016 entries, 0 to 36015
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   gmap_id  36016 non-null  object
 1   user_id  36016 non-null  object
 2   time     36016 non-null  object
 3   rating   36016 non-null  object
 4   text     20950 non-null  object
 5   pics     1364 non-null   object
 6   resp     4720 non-null   object
dtypes: object(7)
memory usage: 1.9+ MB


Since the time and rating columns are not in appropriate data type, changes must be made. The time data will be converted into datetime data type, while the rating data will be converted into integer.

In [382]:
# Convert time from object into datetime data type
df_all["time"] = pd.to_datetime(df_all["time"])

# Convert rating from object into int64
df_all["rating"] = pd.to_numeric(df_all["rating"], errors = "coerce").astype("Int64")

Look at the data information again.

In [383]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36016 entries, 0 to 36015
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   gmap_id  36016 non-null  object        
 1   user_id  36016 non-null  object        
 2   time     36016 non-null  datetime64[ns]
 3   rating   36016 non-null  Int64         
 4   text     20950 non-null  object        
 5   pics     1364 non-null   object        
 6   resp     4720 non-null   object        
dtypes: Int64(1), datetime64[ns](1), object(5)
memory usage: 2.0+ MB


To get another data information, the describe() command can be ran. It will shows data statistics and another information.

In [384]:
df_all.describe()

,time,rating
count,36016,36016.0
mean,2019-02-06 05:24:22.798440960,4.367753
min,2007-06-07 00:00:00,1.0
25%,2018-03-07 23:52:33,4.0
50%,2019-04-09 21:18:30.500000,5.0
75%,2020-02-11 02:02:34.095749888,5.0
max,2021-09-03 00:31:55,5.0
std,NaN,1.138282


In [385]:
# Describe for un-numeric columns
df_all.describe(include = ['O'])

,gmap_id,user_id,text,pics,resp
count,36016,36016,20950,1364,4720
unique,176,34964,19604,1318,4620
top,0x80deab0d284064d9:0xba249d609c190abe,106179618056720671043,Good,[{'url': ['https://lh5.googleusercontent.com/p...,"{'time': 1629747779487, 'text': 'Dear Frederic..."
freq,2298,3,39,2,2


Afterwhich, the data duplicate need to be removed to avoid data inaccuracy.

In [386]:
# Drop all the duplicate from the data
df_all = df_all.drop_duplicates(ignore_index = True)

df_all

,gmap_id,user_id,time,rating,text,pics,resp
0,0x808e32803ca5056d:0x1748a17978e712a3,116363567657011126649,2021-08-21 20:01:03.000,5,I've bought paints here and there coming back ...,<NA>,"{'time': 1629747779487, 'text': 'Dear Frederic..."
1,0x808e32803ca5056d:0x1748a17978e712a3,109312257578567385533,2021-07-21 06:53:03.000,5,Liz was very good salesperson...patient and he...,<NA>,"{'time': 1626990530004, 'text': 'Dear Frank C ..."
2,0x808e32803ca5056d:0x1748a17978e712a3,118432808700092375314,2021-07-01 17:47:57.000,5,I was able to find the paint chips I wanted.,<NA>,"{'time': 1625697160077, 'text': 'Dear Lisa Han..."
3,0x808e32803ca5056d:0x1748a17978e712a3,118011538546540420445,2020-12-10 01:49:29.000,5,I go there every day as a contractor my time i...,<NA>,"{'time': 1607731545910, 'text': 'Dear John Can..."
4,0x808e32803ca5056d:0x1748a17978e712a3,103167319985107391775,2021-08-13 23:01:15.000,5,High quality paints,<NA>,"{'time': 1629222927133, 'text': 'Dear jose san..."
...,...,...,...,...,...,...,...
35267,0x80c2b7dd2ce8c6cb:0x9d551bcaed68b6c0,109708230974290243002,2019-03-14 22:27:34.475,5,<NA>,<NA>,"{'time': 1552660537773, 'text': ""Ana Thank yo..."
35268,0x80c2b7dd2ce8c6cb:0x9d551bcaed68b6c0,101706798456447612925,2021-01-26 00:37:22.816,3,<NA>,<NA>,"{'time': 1611623931823, 'text': 'Jose it looks..."
35269,0x80c2b7dd2ce8c6cb:0x9d551bcaed68b6c0,115802745896517675789,2021-04-03 21:12:11.894,5,<NA>,<NA>,"{'time': 1617715262862, 'text': ""Lalo Thank yo..."
35270,0x80c2b7dd2ce8c6cb:0x9d551bcaed68b6c0,101780044382660069981,2019-11-09 23:49:11.264,4,<NA>,<NA>,"{'time': 1573483390798, 'text': 'Barry So clo..."


The next step is make sure that the review text should be transformed into lower case with no emojis.

In [387]:
# Change the text into lower case
df_all["text"] = df_all["text"].str.lower()

df_all

<ipython-input-387-ab2c45aec26e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_all["text"] = df_all["text"].str.lower()


,gmap_id,user_id,time,rating,text,pics,resp
0,0x808e32803ca5056d:0x1748a17978e712a3,116363567657011126649,2021-08-21 20:01:03.000,5,i've bought paints here and there coming back ...,<NA>,"{'time': 1629747779487, 'text': 'Dear Frederic..."
1,0x808e32803ca5056d:0x1748a17978e712a3,109312257578567385533,2021-07-21 06:53:03.000,5,liz was very good salesperson...patient and he...,<NA>,"{'time': 1626990530004, 'text': 'Dear Frank C ..."
2,0x808e32803ca5056d:0x1748a17978e712a3,118432808700092375314,2021-07-01 17:47:57.000,5,i was able to find the paint chips i wanted.,<NA>,"{'time': 1625697160077, 'text': 'Dear Lisa Han..."
3,0x808e32803ca5056d:0x1748a17978e712a3,118011538546540420445,2020-12-10 01:49:29.000,5,i go there every day as a contractor my time i...,<NA>,"{'time': 1607731545910, 'text': 'Dear John Can..."
4,0x808e32803ca5056d:0x1748a17978e712a3,103167319985107391775,2021-08-13 23:01:15.000,5,high quality paints,<NA>,"{'time': 1629222927133, 'text': 'Dear jose san..."
...,...,...,...,...,...,...,...
35267,0x80c2b7dd2ce8c6cb:0x9d551bcaed68b6c0,109708230974290243002,2019-03-14 22:27:34.475,5,<NA>,<NA>,"{'time': 1552660537773, 'text': ""Ana Thank yo..."
35268,0x80c2b7dd2ce8c6cb:0x9d551bcaed68b6c0,101706798456447612925,2021-01-26 00:37:22.816,3,<NA>,<NA>,"{'time': 1611623931823, 'text': 'Jose it looks..."
35269,0x80c2b7dd2ce8c6cb:0x9d551bcaed68b6c0,115802745896517675789,2021-04-03 21:12:11.894,5,<NA>,<NA>,"{'time': 1617715262862, 'text': ""Lalo Thank yo..."
35270,0x80c2b7dd2ce8c6cb:0x9d551bcaed68b6c0,101780044382660069981,2019-11-09 23:49:11.264,4,<NA>,<NA>,"{'time': 1573483390798, 'text': 'Barry So clo..."


Defining regular expression for the emojis in utf-8.

In [388]:
emoji_pattern = re.compile(
    "["
    "\U0001F600-\U0002F64F"
    "\U0001F300-\U0001F5FF"
    "\U0001F680-\U0001F6FF"
    "\U0001F1E0-\U0001F1FF"
    "\U00002702-\U000027B0"
    "\U000024C2-\U0001F251"

    # Additional emoji ranges not covered in the original pattern
    "\U0001F700-\U0001F77F"  # Alchemical symbols
    "\U0001F780-\U0001F7FF"  # Geometric shapes (extended)
    "\U0001F800-\U0001F8FF"  # Supplemental arrows and symbols
    "\U0001F900-\U0001F9FF"  # Supplemental symbols and pictographs
    "\U0001FA00-\U0001FA6F"  # Symbols for tools, animals, and household objects
    "\U0001FA70-\U0001FAFF"  # Symbols for accessibility and activities
    "\U00002600-\U000026FF"  # Miscellaneous symbols
    "\U00002B50-\U00002B59"  # Star symbols
    "]+"
)

In [389]:
# Delete the emojis from the text
df_all["text"] = df_all["text"].apply(lambda x: emoji_pattern.sub(r'', x) if isinstance(x, str) else x)

df_all

<ipython-input-389-68524eff593b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_all["text"] = df_all["text"].apply(lambda x: emoji_pattern.sub(r'', x) if isinstance(x, str) else x)


,gmap_id,user_id,time,rating,text,pics,resp
0,0x808e32803ca5056d:0x1748a17978e712a3,116363567657011126649,2021-08-21 20:01:03.000,5,i've bought paints here and there coming back ...,<NA>,"{'time': 1629747779487, 'text': 'Dear Frederic..."
1,0x808e32803ca5056d:0x1748a17978e712a3,109312257578567385533,2021-07-21 06:53:03.000,5,liz was very good salesperson...patient and he...,<NA>,"{'time': 1626990530004, 'text': 'Dear Frank C ..."
2,0x808e32803ca5056d:0x1748a17978e712a3,118432808700092375314,2021-07-01 17:47:57.000,5,i was able to find the paint chips i wanted.,<NA>,"{'time': 1625697160077, 'text': 'Dear Lisa Han..."
3,0x808e32803ca5056d:0x1748a17978e712a3,118011538546540420445,2020-12-10 01:49:29.000,5,i go there every day as a contractor my time i...,<NA>,"{'time': 1607731545910, 'text': 'Dear John Can..."
4,0x808e32803ca5056d:0x1748a17978e712a3,103167319985107391775,2021-08-13 23:01:15.000,5,high quality paints,<NA>,"{'time': 1629222927133, 'text': 'Dear jose san..."
...,...,...,...,...,...,...,...
35267,0x80c2b7dd2ce8c6cb:0x9d551bcaed68b6c0,109708230974290243002,2019-03-14 22:27:34.475,5,<NA>,<NA>,"{'time': 1552660537773, 'text': ""Ana Thank yo..."
35268,0x80c2b7dd2ce8c6cb:0x9d551bcaed68b6c0,101706798456447612925,2021-01-26 00:37:22.816,3,<NA>,<NA>,"{'time': 1611623931823, 'text': 'Jose it looks..."
35269,0x80c2b7dd2ce8c6cb:0x9d551bcaed68b6c0,115802745896517675789,2021-04-03 21:12:11.894,5,<NA>,<NA>,"{'time': 1617715262862, 'text': ""Lalo Thank yo..."
35270,0x80c2b7dd2ce8c6cb:0x9d551bcaed68b6c0,101780044382660069981,2019-11-09 23:49:11.264,4,<NA>,<NA>,"{'time': 1573483390798, 'text': 'Barry So clo..."


In [390]:
df_all.shape

(35272, 7)

-------------------------------------

<div class="alert alert-block alert-warning">

## 5.  Writing to Output Files <a class="anchor" name="write"></a>

</div>

The assignment the output files required are CSV file and JSON file. The CSV file contains summary from the data, whilw JSON file contains the detail about the data.

-------------------------------------

<div class="alert alert-block alert-info">
    
### 5.1. Verification of the Generated CSV File <a class="anchor" name="test_xml"></a>

First of all, a new data frame containing the gmap_id, review_count, review_text_count, and response_count need to be created.

In [391]:
# Create the csv data frame
df_csv = df_all.groupby('gmap_id').agg(
    review_count=('gmap_id', 'size'),  # Count the number of review per gmap_id
    review_text_count=('text', lambda x: x.notna().sum()),  # Count the review text
    response_count=('resp', lambda x: x.notna().sum())  # Count the response
).reset_index() # Create index

df_csv

,gmap_id,review_count,review_text_count,response_count
0,0x54cb85d6a549b93b:0xd5c44360bd54363c,179,112,0
1,0x54d15563a771c8b5:0x852b921a4cdd8a01,74,43,0
2,0x54d25224094a28d7:0x943bc32678e13ffe,95,46,8
3,0x8080355286c398bb:0x225acf33673837de,248,182,0
4,0x808327b248e0e835:0xaf2411357a59211e,103,63,55
...,...,...,...,...
171,0x80ea41d4ac2a2bbf:0x658215fdf001eb2a,140,99,127
172,0x80ec146341bbf3b1:0x78af3b2682089309,765,345,5
173,0x80ec58d9d15d1631:0xd064cdb2aefdac02,73,42,14
174,0x80ecc5a98937bf09:0x4c0240bb6f83246b,307,192,2


Then, save the data frame as a csv file in the student_008 working directory.

In [392]:
# Save data frame as csv file
df_csv.to_csv(os.path.join(student_008, 'task1_008.csv'), index=False)

print(f"Successfully saved task1_008.csv")

Successfully saved task1_008.csv


-------------------------------------

<div class="alert alert-block alert-info">
    
### 5.2. Verification of the Generated JSON File <a class="anchor" name="test_xml"></a>

Before making the JSON file, the important functions must be created first.

In [393]:
# Function to decide there is pic or not in the review
def if_pics(pics):
     return 'Y' if pd.notna(pics) and bool(pics) else 'N'

In [394]:
# Function to exctract the dimension of the pics
def extract_dimensions(pics):
    if pd.isna(pics):
        return [] # Empty list if blank

    pics = str(pics) # Make sure the pics are string
    matches = re.findall(r'=w(\d+)-h(\d+)', pics) # Find pattern =wXXX-hXXX with regex
    dimensions = [[width, height] for width, height in matches]

    return dimensions

In [395]:
# Function to decide there is resp or not in the review
def if_resp(resp):
    return 'Y' if pd.notna(resp) and bool(resp) else 'N'

In [396]:
# Funtion to change pd.NA with 'None'
def replace_na_with_none(data):
    if isinstance(data, list):
        return [replace_na_with_none(item) for item in data]
    elif isinstance(data, dict):
        return {key: replace_na_with_none(value) for key, value in data.items()}
    elif pd.isna(data):
        return 'None'
    else:
        return data


In [397]:
# Group review based on gmap_id
grouped = df_all.groupby('gmap_id')

# Make blank list for output
output = []

for gmap_id, group in grouped:
    reviews = []
    for _, row in group.iterrows():
        # Review structure
        review = {
            'user_id': row.get('user_id'),
            'time': row['time'].strftime('%Y-%m-%d %H:%M:%S'),
            'review_rating': row.get('rating'),
            'review_text': row.get('text'),
            'if_pic': if_pics(row.get('pics')),
            'pic_dim': extract_dimensions(row.get('pics')),
            'if_response': if_resp(row.get('resp')),
        }
        reviews.append(review)

    # Earliest and latest date
    earliest_review_date = group['time'].min().strftime('%Y-%m-%d %H:%M:%S')
    latest_review_date = group['time'].max().strftime('%Y-%m-%d %H:%M:%S')

    # JSON structure for every business (gmap_id)
    business_data = {
        'gmap_id': gmap_id,
        'reviews': reviews,
        'earliest_review_date': earliest_review_date,
        'latest_review_date': latest_review_date
    }

    # Append into the output
    output.append(business_data)

# Change null to None in the output
output_cleaned = replace_na_with_none(output)

Then, save the data output as a JSON file in the same student_008 working directory.

In [398]:
# Save the output as JSON file
with open(os.path.join(student_008, 'task1_008.json'), 'w', encoding='utf-8') as f:
    json.dump(output_cleaned, f, indent=2, ensure_ascii=False)

print(f"Successfully saved task1_008.json")

Successfully saved task1_008.json


-------------------------------------

<div class="alert alert-block alert-warning">

## 6. Summary <a class="anchor" name="summary"></a>

</div>

This code efficiently parses Google Map review data from semi-structured text files and an Excel file, extracting key attributes such as user ID, review time, rating, text, and picture dimensions. A notable challenge was handling varied formats of picture URLs and ensuring correct data types, which was addressed by using regular expressions for accurate parsing and normalization. The output is structured in both CSV and JSON formats, adhering to specified requirements while maintaining data integrity through emoji removal and lowercase transformation. Overall, the implementation demonstrates robust data processing techniques in Python.

-------------------------------------

<div class="alert alert-block alert-warning">

## 7. References <a class="anchor" name="Ref"></a>

</div>




1.   <a class="anchor" name="ref-2"></a>  Additional Emojis in Regex, https://www.openai.com/chatgpt, Accessed 19/10/2024.
2.   <a class="anchor" name="ref-3"></a>  Managing Index Order in DataFrames, https://www.openai.com/chatgpt, Accessed 20/10/2024.
3.   <a class="anchor" name="ref-2"></a>  Debugging Techniques, https://www.openai.com/chatgpt, Accessed 20/10/2024.


-------------------------------------